# Canonical Clustering

## $k$ Nearest-Neighbor
### Distance Measure
The $k$ nearest neighbor is based on the simple idea that when a sample is close to other samples, changes are that they share the same label. To know how close one sample is from another sample, there needs to be a notion of close. For this application we will use euclidian distance also known as the $L^2$ norm

$$ \left\| x \right\|_2 \overset{\textrm{def}}= \left( \sum_{i=1}^n x_i^2 \right)^{1/2}$$

### Classification
In classification we would like to assign certain samples to a class. To do so we would like to implement an algorithm that assigns the class based on the classes of the $k$ nearest neighbors. Try to do so in `.code/knn.py`

In [6]:
import sys
sys.path.append('code/')
import knn

import numpy as np

from sklearn import datasets, model_selection
from sklearn.metrics import adjusted_rand_score

# import some data to play with
X, y = datasets.load_iris(return_X_y=True)

# inspect the shape of the data
print(X.shape)
print(y.shape)

# split data into train and test
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)

# and predict
k=10
y_predicted = knn.predict(X_train, X_test, y_train, k)

# evaluate predictions using y_predicted and y_test


(150, 4)
(150,)


## Naive Bayes
#### TODO: probably this exercise is to extensive, rewrite it with using libraries.
### Dataset
Naive Bayes classifier is often used to filter emails for spam, such as in [this article](https://www.aclweb.org/anthology/E03-1059.pdf). For this exercise we will use a dataset containing samples from email messages. This dataset is available on [openml](https://www.openml.org/d/42673) and can be read in using sklearn:

In [7]:
from sklearn import datasets
import numpy as np
spam = datasets.fetch_openml('auml_eml_1_a')

# lets inspect the dataset
np.set_printoptions(precision=3, suppress=True)
print("Dataset Dictionary Keys: \n{0}\n".format(spam.keys()))
print("Dataset Feature Names: \n{0}\n".format(spam['feature_names']))

# print a few examples of spam and non-spam
print("Example spam data entries: \n{0}\n".format(spam['data'][spam['target']==0][:8,:].T))
print("Example spam targets: \n{0}\n".format(spam['target'][spam['target']==0][:8].T))
print("Example non-spam data entries: \n{0}\n".format(spam['data'][spam['target']==1][:8,:].T))
print("Example non-spam targets: \n{0}\n".format(spam['target'][spam['target']==1][:8].T))

Dataset Dictionary Keys: 
dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

Dataset Feature Names: 
['F1_text_html_found', 'F1_number_of_links', 'F1_contains_javascript']

Example spam data entries: 
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 5. 4. 3. 3. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]

Example spam targets: 
[0. 0. 0. 0. 0. 0. 0. 0.]

Example non-spam data entries: 
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]

Example non-spam targets: 
[1. 1. 1. 1. 1. 1. 1. 1.]



This dataset contains 3 features. The first is a binary feature named 'F1_text_html_found', the second is a nominal feature named 'F1_number_of_links', and the last is again a binary feature named 'F1_contains_javascript'. Using these three features, we are going to try to predict the change of a sample being spam. To do so we use:

\begin{align}
P(Y=y_k | X=x_i) = \frac{P(Y=y_k) P(X=x_i | Y=y_k)}{P(X=x_i)}
\end{align}

This is an alternative way of writing:

\begin{align}
P(Y=y_k | X=x_i) = \frac{P(Y=y_k) P(X_1=x_{i,1} \wedge \ldots \wedge X_d=x_{i,d}) | Y=y_k)}{P(X_1=x_{i,1} \wedge \ldots \wedge X_d=x_{i,d})}
\end{align}

The nominator is often omitted, because it scales all classes the same so when comparing classes it does not matter to omit it. Furthermore, it simplifies the calculation. Thus we ultimately we will use:

\begin{align}
P(Y=y_k | X=x_i) = P(Y=y_k) P(X_1=x_{i,1} \wedge \ldots \wedge X_d=x_{i,d}) | Y=y_k)
\end{align}

To use this formula, we first need to define what the probality distribution is. For a binary this is relatively simple. For the counted feature 'F1_number_of_links', we will use a probability distribution based on a histogram with bins with a width of $10$. Implement these probabilities in `code/utils.py`.

In [8]:
import sys
sys.path.append('code/')
import naive_bayes

import pprint
from sklearn.model_selection import train_test_split

X = spam.data
y = spam.target

X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y)

feature_types = ['binary', 'histogram', 'binary']

model = naive_bayes.fit(X_train, y_train, feature_types)

# inspect model
#pprint.pprint(model)

y_predict = naive_bayes.predict(X_test, model)